# Performing landscape level predictions

Mount drive, install package


In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install git+https://github.com/PatBall1/detectree2.git

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PatBall1/detectree2.git to /tmp/pip-req-build-ywt1rdre
  Running command git clone --filter=blob:none --quiet https://github.com/PatBall1/detectree2.git /tmp/pip-req-build-ywt1rdre
  Filtering content: 100% (2/2), 954.89 MiB | 241.17 MiB/s
  Filtering content: 100% (2/2), 954.89 MiB | 74.56 MiB/s, done.
  Resolved https://github.com/PatBall1/detectree2.git to commit 229eafe886c06aad43289d471251cd69e6963e61
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-install-sfsrpwf5/detectron2_7f7a80bb03bb4593967a90caaebe8d48
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-sfsrpwf5/detectron2_7f7a80bb03bb4593967a90caaebe8d48
  Resolved https://github.com/facebookresearch/detectron2.git to commit 072713649a9b0

## Tiling
Tile up the entire orthomosaic to make predictions

In [11]:
import rasterio
### 
site_path = "/content/drive/Shareddrives/detectree2/data/Nouragues"

img_path = site_path + "/rgb/Nouragues_RVB_10cm_0010.tif"
#img_path = site_path + "/rgb/2016/Paracou_RGB_2016_10cm.tif"
tiles_path = site_path + "/tilespred_map10/"
#model_path = "/content/drive/Shareddrives/detectree2/models/220629_ParacouSepilokDanum_JB.pth"

# Read in the tiff file
data = rasterio.open(img_path)


In [12]:
from detectree2.preprocessing.tiling import tile_data
buffer = 20
tile_width = 45
tile_height = 45
tile_data(data, tiles_path, buffer, tile_width, tile_height, dtype_bool = True)

/usr/local/lib/python3.9/dist-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/detectree2/preprocessing/tiling.py:15: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs

## Create full site predictions

### Access pre-trained model

Download pre-trained model

In [9]:
!wget https://github.com/PatBall1/detectree2/raw/master/model_garden/230103_randresize_full.pth

--2023-03-20 16:58:05--  https://github.com/PatBall1/detectree2/raw/master/model_garden/230103_randresize_full.pth
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/PatBall1/detectree2/master/model_garden/230103_randresize_full.pth [following]
--2023-03-20 16:58:05--  https://media.githubusercontent.com/media/PatBall1/detectree2/master/model_garden/230103_randresize_full.pth
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 498209001 (475M) [application/octet-stream]
Saving to: ‘230103_randresize_full.pth’

230103_randresize_f 100%[===================>] 475.13M   390MB/s    in 1.2s  

There might be a cleaner way to do it with `pkg_resources` but I haven't figured this out yet

In [8]:
import pkg_resources
trained_model = pkg_resources.resource_filename('model_garden', "230103_randresize_full.pth")
trained_model

'/usr/local/lib/python3.9/dist-packages/model_garden/230103_randresize_full.pth'

In [7]:
import importlib.resources as pkg_resources
pkg_resources.open_binary("detectree2.model_garden", "230103_randresize_full.pth")

ModuleNotFoundError: ignored

In [13]:
from detectree2.models.train import MyTrainer, setup_cfg

# pretrained model?
#trained_model = "/content/drive/Shareddrives/detectree2/models/220629_ParacouSepilokDanum_JB.pth"
#trained_model = "/content/drive/Shareddrives/detectree2/models/220723_withParacouUAV.pth"
#trained_model = "/content/drive/Shareddrives/detectree2/models/230103_resize_full/model_4.pth"
trained_model = "./230103_randresize_full.pth"

cfg = setup_cfg(update_model=trained_model) # update_model arg can be used to load in trained  model

Make predictions!

In [14]:
from detectree2.models.predict import predict_on_data
from detectron2.engine import DefaultPredictor
#cfg.MODEL.DEVICE='cpu'
predict_on_data(tiles_path, DefaultPredictor(cfg))

/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


/content/drive/Shareddrives/detectree2/data/Nouragues/tilespred_map10/predictions/Prediction_Nouragues_RVB_10cm_0010_312202_445286_45_20_2972.json
/content/drive/Shareddrives/detectree2/data/Nouragues/tilespred_map10/predictions/Prediction_Nouragues_RVB_10cm_0010_312382_445646_45_20_2972.json
/content/drive/Shareddrives/detectree2/data/Nouragues/tilespred_map10/predictions/Prediction_Nouragues_RVB_10cm_0010_311932_445601_45_20_2972.json
/content/drive/Shareddrives/detectree2/data/Nouragues/tilespred_map10/predictions/Prediction_Nouragues_RVB_10cm_0010_312022_445511_45_20_2972.json
/content/drive/Shareddrives/detectree2/data/Nouragues/tilespred_map10/predictions/Prediction_Nouragues_RVB_10cm_0010_312157_445331_45_20_2972.json
/content/drive/Shareddrives/detectree2/data/Nouragues/tilespred_map10/predictions/Prediction_Nouragues_RVB_10cm_0010_312157_445691_45_20_2972.json
/content/drive/Shareddrives/detectree2/data/Nouragues/tilespred_map10/predictions/Prediction_Nouragues_RVB_10cm_0010_3

Here we reproject predictions into geographical space

In [ ]:
from detectree2.models.outputs import project_to_geojson
project_to_geojson(tiles_path, tiles_path + "predictions/", tiles_path + "predictions_geo/")

Prediction_Nouragues_RVB_10cm_0010_311977_445331_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312337_445511_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312112_445736_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312247_445421_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312112_445331_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312022_445691_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312112_445421_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_311932_445376_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312112_445466_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312157_445421_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312292_445286_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312022_445376_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_311932_445736_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312067_445466_45_20_2972.json
Prediction_Nouragues_RVB_10cm_0010_312112_445556_45_20_2972.json
Prediction_Nouragues_RVB_

Combine predictions into full output

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from detectree2.models.outputs import stitch_crowns, clean_crowns
#name = "Paracou"
folder = tiles_path+ "/predictions_geo"
crowns = stitch_crowns(folder, 1)


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.9/dist-packages/detectree2/models/outputs.py:275: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crowns = crowns.append(crowns_tile)
/usr/local/lib/python3.9/dist-packages/detectree2/models/outputs.py:275: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crowns = crowns.append(crowns_tile)
/usr/local/lib/python3.9/dist-packages/detectree2/models/outputs.py:275: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crowns = crowns.append(crowns_tile)
/usr/local/lib/python3.9/dist-packages/detectree2/models/outputs.py:275: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  crowns = 

Remove invalid geometries before cleaning. Might be a way to fix them rather than remove all together but I think the ones with invalid geometries tend to be bad crowns anyway.

In [ ]:
crowns = crowns[crowns.is_valid]
# Simplify crowns to help with editing 
crowns = crowns.set_geometry(crowns.simplify(0.3))
crowns

,Confidence_score,geometry
0,0.684301,"POLYGON ((311972.128 445336.473, 311970.128 44..."
1,0.665960,"POLYGON ((312006.128 445348.773, 312003.928 44..."
2,0.610145,"POLYGON ((311997.128 445333.073, 311993.528 44..."
3,0.569785,"POLYGON ((312016.628 445359.373, 312013.228 44..."
4,0.495516,"POLYGON ((311967.728 445355.473, 311965.928 44..."
...,...,...
7929,0.141906,"POLYGON ((312397.428 445289.473, 312394.128 44..."
7930,0.141382,"POLYGON ((312418.828 445298.973, 312417.728 44..."
7931,0.141144,"POLYGON ((312371.528 445320.573, 312369.528 44..."
7932,0.139175,"POLYGON ((312411.628 445318.273, 312408.128 44..."


Clean overlapping crowns. For significant overlaps, the most confident crown is selected. IoU can be varied based on how strict you want the filtering to be.

In [ ]:
crowns = clean_crowns(crowns, 0.4)
crowns.to_file(site_path + "/crowns_map2.gpkg")

0 / 7825 cleaned
1000 / 7825 cleaned
2000 / 7825 cleaned
3000 / 7825 cleaned
4000 / 7825 cleaned
5000 / 7825 cleaned
6000 / 7825 cleaned
7000 / 7825 cleaned
